# Getting Started (with Google Colab)
- In the [Google Colab](https://colab.research.google.com/) menu, go to File -> Open notebook
- In the dialogue that appears, click the GitHub tab
- Enter this repo as your GitHub URL, i.e., https://github.com/davedgd/studentcourse
- Click the search icon (or hit enter)
- Select the notebook you want to open to get started (e.g., studentcourse-llm.ipynb)
- **Note:** You should also change the runtime type to one with a GPU...
    - In the Google Colab menu, go to Runtime -> Change runtime type
    - Under 'Hardware accelerator' choose at least a 'T4 GPU' (or better)
    - Click the Save button
- To run the notebook, in the Google Colab menu, go to Runtime -> Run all
- If you see a 'Warning: This notebook was not authored by Google' popup, click the 'Run anyway' button
- Wait patiently, as it make take a few minutes to install the packages and run the notebook
- **Note:** You may see occasional error messages, particularly when installing/loading pip packages -- you can generally safely ignore these (as long as the notebook runs)

In [1]:
# note: you can generally ignore ERROR messages regarding "pip's dependency resolver"
!pip install transformers huggingface_hub[hf_transfer] autoawq -U -q --progress-bar off

import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

  Preparing metadata (setup.py) ... done


In [2]:
# for more about Qwen2.5-Coder, see:
# https://qwenlm.github.io/blog/qwen2.5-coder-family/

# adapted from https://qwen.readthedocs.io/en/latest/inference/chat.html

from transformers import pipeline

pipe = pipeline('text-generation',
                model = 'Qwen/Qwen2.5-Coder-7B-Instruct-AWQ',
                device_map = 'auto')

# depending on available VRAM, you may be able to use larger models:
# https://huggingface.co/collections/Qwen/qwen25-coder-66eaa22e6f99801bf65b0c2f

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
schema = '''
CREATE TABLE major (
	major VARCHAR(255),
	school VARCHAR(255),
	CONSTRAINT PK_major PRIMARY KEY (major)
	);

CREATE TABLE student (
	studentid INT,
	name VARCHAR(255),
	major VARCHAR(255),
	year INT,
	CONSTRAINT PK_student PRIMARY KEY (studentid),
	CONSTRAINT FK_student_to_major FOREIGN KEY (major)
		REFERENCES major(major)
	);

CREATE TABLE course (
	courseid INT,
	topic VARCHAR(255),
	title VARCHAR(255),
	CONSTRAINT PK_course PRIMARY KEY (courseid)
	);

CREATE TABLE takes (
	studentid INT,
	courseid INT,
	date date,
	CONSTRAINT PK_takes PRIMARY KEY (studentid, courseid),
	CONSTRAINT FK_takes_to_student FOREIGN KEY (studentid)
		REFERENCES student(studentid),
	CONSTRAINT FK_takes_to_course FOREIGN KEY (courseid)
		REFERENCES course(courseid)
	);

INSERT INTO major (major, school) VALUES ('COMM', 'Commerce');
INSERT INTO major (major, school) VALUES ('CS', 'Engineering');
INSERT INTO major (major, school) VALUES ('SE', 'Engineering');

INSERT INTO student VALUES (100, 'Lucy', 'COMM', 4);
INSERT INTO student VALUES (140, 'Brian', 'CS', 2);
INSERT INTO student VALUES (130, 'Lindsey', 'SE', 4);

INSERT INTO course VALUES (3200, 'RDBMS', 'Intro...');
INSERT INTO course VALUES (7450, 'Stats', 'Adv...');
INSERT INTO course VALUES (3600, 'Big Data', 'Intro...');
INSERT INTO course VALUES (3460, 'Finance', 'Intro...');
INSERT INTO course VALUES (4200, 'Global', 'Adv...');
INSERT INTO course VALUES (3100, 'Global', 'Intro...');

INSERT INTO takes VALUES (100, 3200, '2016-11-01');
INSERT INTO takes VALUES (100, 7450, '2016-11-01');
INSERT INTO takes VALUES (100, 4200, '2016-12-01');
INSERT INTO takes VALUES (140, 3600, '2016-12-01');
INSERT INTO takes VALUES (140, 3460, '2016-12-01');
INSERT INTO takes VALUES (140, 4200, '2016-01-01');
INSERT INTO takes VALUES (130, 3100, '2016-12-01');
INSERT INTO takes VALUES (130, 3600, '2016-12-01');
INSERT INTO takes VALUES (130, 3200, '2016-12-01');

CREATE TABLE engmajor (
	major VARCHAR(255),
	school VARCHAR(255),
	CONSTRAINT PK_engmajor PRIMARY KEY (major)
	);

INSERT INTO engmajor (major, school) VALUES ('CS', 'Engineering');
INSERT INTO engmajor (major, school) VALUES ('SE', 'Engineering');

CREATE TABLE students (
	studentid INT,
	name VARCHAR(255),
	major VARCHAR(255),
	year INT,
	CONSTRAINT PK_students PRIMARY KEY (studentid),
	CONSTRAINT FK_students_to_major FOREIGN KEY (major)
		REFERENCES major(major)
	);

CREATE TABLE studentemp (
	studentident INT,
	hourlysalary INT,
	CONSTRAINT PK_studentemp PRIMARY KEY (studentident)
	);

INSERT INTO studentemp VALUES (150, 12);
INSERT INTO studentemp VALUES (130, 15);
INSERT INTO studentemp VALUES (190, 20);

CREATE TABLE morestudent (
	studentid INT,
	name VARCHAR(255),
	major VARCHAR(255),
	year INT,
	CONSTRAINT PK_morestudent PRIMARY KEY (studentid, major),
	CONSTRAINT FK_morestudent_to_major FOREIGN KEY (major)
		REFERENCES major(major)
	);

INSERT INTO morestudent VALUES (150, 'Tim', 'COMM', 3);
INSERT INTO morestudent VALUES (140, 'Brian', 'CS', 2);
INSERT INTO morestudent VALUES (160, 'Will', 'COMM', 2);
'''

prompt = '''
### Task
Consider the database schema below and answer the question that follows.

### Database Schema
This query will run on a database whose schema is represented in this string:

```sql{schema}```

### SQL
Given the database schema, here is the SQL query that answers `{question}`:
'''

In [4]:
question = 'How many courses are each of the students in the STUDENT table currently taking?'

# the default system message will be used
messages = [{'role': 'user', 'content': prompt.format(question = question,
                                                      schema = schema)}]

response_message = pipe(messages, max_new_tokens = 512)[0]['generated_text'][-1]

print(response_message['content'])

To determine how many courses each student in the `STUDENT` table is currently taking, we can use the following SQL query:

```sql
SELECT s.studentid, s.name, COUNT(t.courseid) AS number_of_courses
FROM student s
JOIN takes t ON s.studentid = t.studentid
GROUP BY s.studentid, s.name;
```

This query works as follows:
- It joins the `student` table (`s`) with the `takes` table (`t`) on the `studentid` field to get the courses taken by each student.
- It groups the results by `studentid` and `name` to ensure that the count of courses is calculated for each individual student.
- It uses the `COUNT` function to count the number of `courseid` entries for each student, which represents the number of courses they are currently taking.
- The result includes the `studentid`, `name`, and the `number_of_courses` for each student.
